# Data mining techniques: Assignment 2

This environment makes use of Python version 3.6.3

In [1]:
# Used for data frame and numerical calculations
import pandas as pd
import numpy as np

# Only for progress bar and print output (not needed for report)
import sys
import tqdm

# Used for price and star rankings
from scipy.stats import rankdata

# Set data variables

In [2]:
# Set paths to the data sets used
fullPath = "/home/kevin/data_mining/data/assignment2/initial_train.csv"
trainPath = "/home/kevin/data_mining/data/assignment2/split_data_dm/train.csv"
validPath = "/home/kevin/data_mining/data/assignment2/split_data_dm/validation.csv"
testPath = "/home/kevin/data_mining/data/assignment2/split_data_dm/test.csv"

# Get the names of all columns
fullcols = list(pd.read_csv(fullPath,nrows=1).columns.values)
traincols = list(pd.read_csv(trainPath,nrows=1).columns.values)
validcols = list(pd.read_csv(validPath,nrows=1).columns.values)
testcols = list(pd.read_csv(testPath,nrows=1).columns.values)

# Feature Engineering

## Extract relevant date_time values

In [3]:
def extract_date_values(df):
    "Extracts relevant date values"
    df_dates = pd.to_datetime(df["date_time"])

    time_df = pd.DataFrame({"date_year":df_dates.dt.year,
                            "date_month":df_dates.dt.month,
                            "date_hour":df_dates.dt.hour})

    return time_df

def retrieve_relevance(df):
    "Retrieves the relevance from booking_bool and click_bool"
    relevance = np.zeros(len(df),dtype=int)
    
    relevance[df["booking_bool"].values == 1] = 5
    relevance[(df["click_bool"].values == 1) & (df["booking_bool"].values == 0)] = 1
    
    return relevance

def extract_starrating_diff(df):
    return np.abs(df["visitor_hist_starrating"] - df["prop_starrating"])

def extract_usd_diff(df):
    return np.abs(df["visitor_hist_adr_usd"] - df["price_usd"])

def extract_monotonic_feature(df,attribute,booked_mean):
    return np.abs(df[attribute] - booked_mean)

def extract_rank_feature(df,attribute,ascending=True):
    # Dense ranking allows duplicate rank numbers (ranking does not add up to length of srch_id rows)
    return df.groupby('srch_id')[attribute].rank(ascending=ascending,method="dense").values.astype(int)

def engineer_dataset(path,cols,data=None):
    """
    Does feature engineering on dataset and 
    returns the modified data frame
    """
    
    if data is None:
        df = pd.read_csv(path,usecols=cols)
    else:
        df = data

    print("Data loaded")

    # Get date values and insert in data frame
    time_df = extract_date_values(df)

    for column in time_df.columns.values:
        df[column] = time_df[column]

    del time_df

    print("Time values extracted")

    # Drop the date_time attribute
    df.drop('date_time', axis=1, inplace=True)

    # Insert diff attributes in data frame
    df["starrating_diff"] = extract_starrating_diff(df)
    df["usd_diff"] = extract_usd_diff(df)

    print("Difference attributes extracted")

    # Insert rank attributes in data frame
    rank_attributes = ["price_usd","prop_starrating"]
    rank_names = ["price_rank","star_rank"]
    ascend = [True,False]
    for n in range(len(rank_attributes)):
        df[rank_names[n]] = extract_rank_feature(df,rank_attributes[n],ascending=ascend[n])

    print("Rank attributes extracted")

    # Insert relevance in data frame
    #df["relevance"] = retrieve_relevance(df)
    
    return df

## Export engineered full data set

In [4]:
#df = engineer_dataset(fullPath,fullcols)

#df.head(10)

In [5]:
# Write to a CSV file
#df.to_csv("/home/kevin/data_mining/data/assignment2/full_train.csv",sep=',',index=False)

# Missing values imputation

In [6]:
# Initialize missing values dictionary
imp = {}

## Origin-destination distance

In [7]:
# Get statistics from train data for imputing splits/final test set

attributes = ["visitor_location_country_id","prop_country_id","orig_destination_distance"]

# Might want to use both train and valid for imputation
df = pd.read_csv(trainPath,usecols=attributes)

max_n = max(np.max(df["visitor_location_country_id"]),np.max(df["prop_country_id"]))

mean_distances = np.empty((max_n,max_n))
mean_distances[:,:] = np.nan

dist_sum = np.zeros((max_n,max_n))
dist_count = np.zeros((max_n,max_n))

dist_nans = np.isnan(df["orig_destination_distance"])

print("Number of NaNs before:",np.sum(dist_nans))

sys.stdout.flush()

i = np.maximum(df["visitor_location_country_id"],df["prop_country_id"]) - 1
j = np.minimum(df["visitor_location_country_id"],df["prop_country_id"]) - 1

# Fill NAs with distance pairs
with tqdm.tqdm(total=np.sum(~dist_nans)) as pbar:
    for n in np.arange(len(df))[~dist_nans]:
        dist_sum[i[n],j[n]] += df.loc[n,"orig_destination_distance"]
        dist_count[i[n],j[n]] += 1
        
        pbar.update()

mean_indices = (dist_count != 0)
mean_distances[mean_indices] = dist_sum[mean_indices] / dist_count[mean_indices]

df.loc[dist_nans,"orig_destination_distance"] = mean_distances[i[dist_nans],j[dist_nans]]
    
dist_nans = np.isnan(df["orig_destination_distance"])
print("Number of NaNs after pairs:",np.sum(dist_nans))

# Fill NAs with medians per visitor location ID
visitor_medians = np.zeros(np.max(df["visitor_location_country_id"]))
visitor_medians[:] = np.nan

vis_loc_ids = df["visitor_location_country_id"] - 1
for n in range(len(visitor_medians)):
    vals = (vis_loc_ids == n) & ~dist_nans
    nans = (vis_loc_ids == n) & dist_nans
    
    dist_median = np.median(df.loc[vals,"orig_destination_distance"])
    df.loc[nans,"orig_destination_distance"] = dist_median
    visitor_medians[n] = dist_median

dist_nans = np.isnan(df["orig_destination_distance"])
print("Number of NaNs after visitor ID imputation:",np.sum(dist_nans))
    
# Fill remaining NAs with the median of all distances
median_distance = np.median(df.loc[~dist_nans,"orig_destination_distance"])
df.loc[dist_nans,"orig_destination_distance"] = median_distance

print("Number of NaNs after:",np.sum(np.isnan(df["orig_destination_distance"])))

imp["mean_distances"] = mean_distances
imp["visitor_medians"] = visitor_medians
imp["median_distance"] = median_distance

del df

Number of NaNs before: 1127331


100%|██████████| 2343504/2343504 [03:21<00:00, 11610.61it/s]


Number of NaNs after pairs: 331853


/home/kevin/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/kevin/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Number of NaNs after visitor ID imputation: 15310
Number of NaNs after: 0


# Create the imputer

In [8]:
# Get statistics or set values for imputation

attributes = ["prop_location_score2","srch_query_affinity_score","prop_starrating","booking_bool"]

# Might want to use both train and valid for statistics
df = pd.read_csv(trainPath,usecols=attributes)

imp["prop_review_score"] = 0

imp["usd_diff"] = 0

imp["starrating_diff"] = 0

# For comp attributes
imp["comp"] = 0

nans = np.isnan(df["prop_location_score2"])
imp["prop_location_score2"] = np.percentile(df.loc[~nans,"prop_location_score2"],25)

nans = np.isnan(df["srch_query_affinity_score"])
imp["srch_query_affinity_score"] = np.percentile(df.loc[~nans,"srch_query_affinity_score"],25)

booked = (df["booking_bool"] == 1)
imp["starrating_booked_mean"] = np.mean(df.loc[booked,"prop_starrating"])

del df

# Export train data to CSV

In [9]:
def impute_dataset(path,cols,imp,omitted=[],data=None):
    """
    Impute the missing values of the dataset
    and engineer variables from train statistics
    """

    if data is None:
        attributes = [elem for elem in cols if elem not in omitted]
        df = pd.read_csv(path,usecols=attributes)
    else:
        df = data

    # Impute origin-distance values

    dist_nans = np.isnan(df["orig_destination_distance"])
    i = np.maximum(df["visitor_location_country_id"],df["prop_country_id"]) - 1
    j = np.minimum(df["visitor_location_country_id"],df["prop_country_id"]) - 1
    df.loc[dist_nans,"orig_destination_distance"] = imp["mean_distances"][i[dist_nans],j[dist_nans]]
    
    dist_nans = np.isnan(df["orig_destination_distance"])
    vis_loc_ids = df["visitor_location_country_id"] - 1
    for n in range(len(visitor_medians)):
        nans = (vis_loc_ids == n) & dist_nans
        df.loc[nans,"orig_destination_distance"] = imp["visitor_medians"][n]
    
    dist_nans = np.isnan(df["orig_destination_distance"])
    df.loc[dist_nans,"orig_destination_distance"] = imp["median_distance"]
    
    # Impute the rest of the missing values
    
    nans = np.isnan(df["prop_review_score"])
    df.loc[nans,"prop_review_score"] = imp["prop_review_score"]
    
    nans = np.isnan(df["usd_diff"])
    df.loc[nans,"usd_diff"] = imp["usd_diff"]
    
    nans = np.isnan(df["starrating_diff"])
    df.loc[nans,"starrating_diff"] = imp["starrating_diff"]

    nans = np.isnan(df["prop_location_score2"])
    df.loc[nans,"prop_location_score2"] = imp["prop_location_score2"]

    nans = np.isnan(df["srch_query_affinity_score"])
    df.loc[nans,"srch_query_affinity_score"] = imp["srch_query_affinity_score"]

    for n in range(1,9):
        nans = np.isnan(df["comp%i_rate" % (n)])
        df.loc[nans,"comp%i_rate" % (n)] = imp["comp"]

        nans = np.isnan(df["comp%i_inv" % (n)])
        df.loc[nans,"comp%i_inv" % (n)] = imp["comp"]

        nans = np.isnan(df["comp%i_rate_percent_diff" % (n)])
        df.loc[nans,"comp%i_rate_percent_diff" % (n)] = imp["comp"]
    
    # Engineer monotonic variables
    booked_mean = imp["starrating_booked_mean"]
    df["prop_starrating_monotonic"] = extract_monotonic_feature(df,"prop_starrating",booked_mean)
    
    return df

# Export data to CSV

## Train data

In [10]:
train = impute_dataset(trainPath,traincols,imp)

print("NaNs: %i" % np.sum(train.isnull().values))

train.head()

NaNs: 9962268


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,date_year,starrating_diff,usd_diff,price_rank,star_rank,relevance,room_count,persons_count,ignored_bool,prop_starrating_monotonic
0,49766,5,219,NaN,NaN,219,10827,0,4.0,1,...,2013,0.0,0.0,2,4,0,522,18,1,3.3131
1,49766,5,219,NaN,NaN,219,33591,4,4.0,1,...,2013,0.0,0.0,14,1,0,522,18,1,0.6869
2,49766,5,219,NaN,NaN,219,43888,3,4.0,1,...,2013,0.0,0.0,7,2,0,522,18,1,0.3131
3,49766,5,219,NaN,NaN,219,46317,3,3.5,1,...,2013,0.0,0.0,6,2,0,522,18,1,0.3131
4,49766,5,219,NaN,NaN,219,47864,4,4.0,0,...,2013,0.0,0.0,16,1,0,522,18,1,0.6869


In [11]:
train.to_csv("/home/kevin/data_mining/data/assignment2/split_data_dm/mod_train.csv",sep=',',index=False)

## Validation data

In [13]:
valid = impute_dataset(validPath,validcols,imp)

print("NaNs: %i" % np.sum(valid.isnull().values))

valid.head()

NaNs: 2134907


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,date_year,starrating_diff,usd_diff,price_rank,star_rank,relevance,room_count,persons_count,ignored_bool,prop_starrating_monotonic
0,49766,5,219,NaN,NaN,219,6642,3,4.0,0,...,2013,0.0,0.0,9,2,0,522,18,1,0.3131
1,49766,5,219,NaN,NaN,219,21167,2,3.5,0,...,2013,0.0,0.0,11,3,0,522,18,1,1.3131
2,49766,5,219,NaN,NaN,219,45731,3,4.5,0,...,2013,0.0,0.0,9,2,0,522,18,1,0.3131
3,49766,5,219,NaN,NaN,219,104310,3,3.5,0,...,2013,0.0,0.0,10,2,0,522,18,1,0.3131
4,49766,5,219,NaN,NaN,219,108197,3,4.0,1,...,2013,0.0,0.0,4,2,1,522,18,0,0.3131


In [14]:
valid.to_csv("/home/kevin/data_mining/data/assignment2/split_data_dm/mod_valid.csv",sep=',',index=False)

## Test data

In [16]:
test = impute_dataset(testPath,testcols,imp)

print("NaNs: %i" % np.sum(test.isnull().values))

test.head()

NaNs: 2134622


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,date_year,starrating_diff,usd_diff,price_rank,star_rank,relevance,room_count,persons_count,ignored_bool,prop_starrating_monotonic
0,1,12,187,NaN,NaN,219,893,3,3.5,1,...,2013,0.0,0.0,3,2,0,492,36,1,0.3131
1,1,12,187,NaN,NaN,219,10404,4,4.0,1,...,2013,0.0,0.0,14,1,0,492,36,1,0.6869
2,1,12,187,NaN,NaN,219,21315,3,4.5,1,...,2013,0.0,0.0,15,2,0,492,36,1,0.3131
3,1,12,187,NaN,NaN,219,27348,2,4.0,1,...,2013,0.0,0.0,22,3,0,492,36,1,1.3131
4,1,12,187,NaN,NaN,219,29604,4,3.5,1,...,2013,0.0,0.0,11,1,0,492,36,1,0.6869


In [17]:
test.to_csv("/home/kevin/data_mining/data/assignment2/split_data_dm/mod_test.csv",sep=',',index=False)

# Real test engineer & impute

In [11]:
realTestPath = "/home/kevin/data_mining/data/assignment2/real_test_with_var.csv"
realtestcols = list(pd.read_csv(realTestPath,nrows=1).columns.values)

realtest = engineer_dataset(realTestPath,realtestcols)

realtest = impute_dataset(None,None,imp,data=realtest)

print(len(realtest))

realtest.to_csv("/home/kevin/data_mining/data/assignment2/real_test_complete.csv",sep=',',index=False)

Data loaded
Time values extracted
Difference attributes extracted
Rank attributes extracted


In [12]:
realtest.head(10)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,room_count,persons_count,date_hour,date_month,date_year,starrating_diff,usd_diff,price_rank,star_rank,prop_starrating_monotonic
0,1,24,216,NaN,NaN,219,3180,3,4.5,1,...,508,18,15,2,2013,0.0,0.0,19,2,0.3131
1,1,24,216,NaN,NaN,219,5543,3,4.5,1,...,508,18,15,2,2013,0.0,0.0,18,2,0.3131
2,1,24,216,NaN,NaN,219,14142,2,3.5,1,...,508,18,15,2,2013,0.0,0.0,3,3,1.3131
3,1,24,216,NaN,NaN,219,22393,3,4.5,1,...,508,18,15,2,2013,0.0,0.0,21,2,0.3131
4,1,24,216,NaN,NaN,219,24194,3,4.5,1,...,508,18,15,2,2013,0.0,0.0,10,2,0.3131
5,1,24,216,NaN,NaN,219,28181,3,4.5,1,...,508,18,15,2,2013,0.0,0.0,12,2,0.3131
6,1,24,216,NaN,NaN,219,34263,3,4.5,1,...,508,18,15,2,2013,0.0,0.0,10,2,0.3131
7,1,24,216,NaN,NaN,219,37567,2,4.5,0,...,508,18,15,2,2013,0.0,0.0,5,3,1.3131
8,1,24,216,NaN,NaN,219,50162,2,3.5,1,...,508,18,15,2,2013,0.0,0.0,4,3,1.3131
9,1,24,216,NaN,NaN,219,54937,3,4.0,1,...,508,18,15,2,2013,0.0,0.0,11,2,0.3131
